# For likeability experiment and turing test

In [2]:
from PIL import Image
import os
import pandas as pd
import sys
import random
import numpy as np

In [3]:
csv_dir = './datasets/processed/likeability'
csv_files = os.listdir(csv_dir)
csvs = {k:pd.read_csv(os.path.join(csv_dir, k)) for k in csv_files}

In [4]:
to_keep = ['highest_NN_dist', 'lowest_NN_dist', 'highest_shape_entropy', 'random']

In [5]:
files = ['StyleGAN1.csv', 'StyleCAN1.csv', 'StyleGAN2.csv', 'StyleCAN2.csv', 'StyleCWAN1.csv', 'StyleCWAN2.csv']

### Table 7

In [6]:
for file in files:
    print(file)
    if file == 'StyleGAN2.csv' or file == 'StyleCAN2.csv':
        name = 'Input.image'
        col =  'Input.mean'
        turing = 'Input.is_generated_by_artist'
    else:
        name = 'Input.kn_img_url'
        col =  'Answer.Q1_answer'
        turing = 'Answer.Q2_answer'
    for grp in to_keep:
        mean_rating = csvs[file][[name,col]].groupby(name).mean().filter(like=grp,axis=0).sort_values(by=col,ascending=True).mean()
        print(grp,': ', mean_rating.values)
    print('Turing Score: ',csvs[file][turing].mean())
    print('Mean Score:',csvs[file][col].mean())
    print('Score Std: ',csvs[file][col].std())

StyleGAN1.csv
highest_NN_dist :  [3.0695]
lowest_NN_dist :  [3.367]
highest_shape_entropy :  [3.002]
random :  [3.067]
Turing Score:  0.553125
Mean Score: 3.1263750000000003
Score Std:  0.5899518772057144
StyleCAN1.csv
highest_NN_dist :  [3.0175]
lowest_NN_dist :  [3.6185]
highest_shape_entropy :  [3.054]
random :  [3.117]
Turing Score:  0.5655449522852838
Mean Score: 3.201204214751631
Score Std:  1.140773426665542
StyleGAN2.csv
highest_NN_dist :  [2.8955]
lowest_NN_dist :  [3.309]
highest_shape_entropy :  [2.794]
random :  [3.09]
Turing Score:  0.538
Mean Score: 3.022125
Score Std:  0.6715409280036415
StyleCAN2.csv
highest_NN_dist :  [3.268]
lowest_NN_dist :  [3.3445]
highest_shape_entropy :  [3.1125]
random :  [3.2095]
Turing Score:  0.577
Mean Score: 3.233625
Score Std:  0.6124546184860168
StyleCWAN1.csv
highest_NN_dist :  [3.1555]
lowest_NN_dist :  [3.673]
highest_shape_entropy :  [3.1525]
random :  [3.179]
Turing Score:  0.5863110216406643
Mean Score: 3.2913108990457056
Score Std:

# Images selected for the figure, asc is used to get least and most likeable and artisitic figures. To view the most and least likeable image from all 4 groups, change the index of to_keep from 0 - 3. For Turing test, change 'col' variable to 'turing'

In [7]:
acs=True

### For turing test, change col to turing
#### csvs[files[2]][[name,turing]].groupby(name).mean().filter(like=to_keep[0],axis=0).sort_values(by=turing,ascending=acs)

In [9]:
csvs[files[1]][[name,col]].groupby(name).mean().filter(like=to_keep[0],axis=0).sort_values(by=col,ascending=acs)

,Answer.Q1_answer
Input.kn_img_url,
https://crw2020.s3-us-west-2.amazonaws.com/styleGAN-human-eval/CAN/highest_NN_dist1503.png,1.4
https://crw2020.s3-us-west-2.amazonaws.com/styleGAN-human-eval/CAN/highest_NN_dist530.png,2.0
https://crw2020.s3-us-west-2.amazonaws.com/styleGAN-human-eval/CAN/highest_NN_dist869.png,2.0
https://crw2020.s3-us-west-2.amazonaws.com/styleGAN-human-eval/CAN/highest_NN_dist131.png,2.2
https://crw2020.s3-us-west-2.amazonaws.com/styleGAN-human-eval/CAN/highest_NN_dist1432.png,2.2
...,...
https://crw2020.s3-us-west-2.amazonaws.com/styleGAN-human-eval/CAN/highest_NN_dist140.png,3.8
https://crw2020.s3-us-west-2.amazonaws.com/styleGAN-human-eval/CAN/highest_NN_dist949.png,3.8
https://crw2020.s3-us-west-2.amazonaws.com/styleGAN-human-eval/CAN/highest_NN_dist1437.png,3.8


## Likeability per emotion

### select files from the follwing 'StyleGAN2.csv', 'StyleCAN2.csv', 'StyleCWAN1.csv', and 'StyleCWAN2.csv' 

In [10]:
file = 'StyleGAN2.csv'

In [11]:
if file == 'StyleCWAN1.csv':
    dataset = './datasets/raw/gen_emotions_cwan.csv'
    emo_csv = pd.read_csv(dataset)       
if file == 'StyleCWAN2.csv':
    dataset = './datasets/raw/emotions_cwan2.csv'
    emo_csv = pd.read_csv(dataset)   

In [12]:
if file == 'StyleCWAN1.csv' or file =='StyleCWAN2.csv':
    name = 'Input.kn_img_url'
    col =  'Answer.Q1_answer'
else:
    name = 'Input.image'
    col =  'Input.mean'

In [13]:
def get_name_emo(row):
    name = row['art_url'].split('/')[-1]
    return name

def get_name_like(row):
    name = row['Input.kn_img_url'].split('/')[-1]
    return name

In [14]:
if file == 'StyleCWAN1.csv':
    emo_csv = emo_csv[emo_csv['art_url'].str.contains('cwan1') == True]
if file == 'StyleCWAN2.csv':
    emo_csv = emo_csv[emo_csv['art_url'].str.contains('sg2') == True]

In [15]:
if file == 'StyleCWAN1.csv' or file =='StyleCWAN2.csv':
    emo_csv['name'] = emo_csv.apply(get_name_emo, axis=1)
    csvs[file]['name'] = csvs[file].apply(get_name_like, axis=1)
    emo_csv_temp = emo_csv.groupby('name').mean()
    csvs_temp = csvs[file].groupby('name').mean()
    df_comb = pd.merge(csvs_temp, emo_csv_temp, on='name')

In [16]:
if file == 'StyleCWAN1.csv' or file =='StyleCWAN2.csv':
    e1, e2, e3, e4, e5, e6, e7, e8, e9 = ['Amusement', 'Anger', 'Awe', 'Contentment', 'Disgust', 'Excitement', 'Fear', 'Sadness', 'Something Else']
else:
    e1, e2, e3, e4, e5, e6, e7, e8, e9 = 'Answer.choice.Amusement','Answer.choice.Anger', 'Answer.choice.Awe', 'Answer.choice.Contentment', 'Answer.choice.Disgust', 'Answer.choice.Excitement','Answer.choice.Fear', 'Answer.choice.Sadness','Answer.choice.Something Else'

In [17]:
sel_emo = [e1, e2, e3, e4, e5, e6, e7, e8, e9]
if file == 'StyleCWAN1.csv' or file =='StyleCWAN2.csv':
    for i in sel_emo:
        print(i,': ',df_comb[[i, col]][df_comb[[i, col]][i]!=0].mean().values[1])
else:
    for i in sel_emo:
        print(i,': ',csvs[file][[i, col]][csvs[file][[i, col]][i]!=0].mean().values[1])        

Answer.choice.Amusement :  2.9603174603174605
Answer.choice.Anger :  2.764705882352941
Answer.choice.Awe :  3.1650826446280993
Answer.choice.Contentment :  3.29760348583878
Answer.choice.Disgust :  2.708387096774193
Answer.choice.Excitement :  3.180891719745223
Answer.choice.Fear :  2.86984126984127
Answer.choice.Sadness :  2.914840182648402
Answer.choice.Something Else :  2.8592391304347826


### The polar chart from Figure 12 can be found [here](https://public.flourish.studio/visualisation/10095550/).